## EMDE
Experiments with Efficient Manifold Density Estimation - based on the paper
https://arxiv.org/abs/2006.01894

In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import scipy.sparse as ssp


def EMDE_transform(K, N, item_vectors):
    """takes a and array of embedding vectors and 
    returns a sparse array of their sketches
    """
    n_items, d = item_vectors.shape
    shallow_sketches = []
    for _ in range(N):
        # first chose K vectors at random - these are the normal vectors to the K hyperplanes
        random_vectors = np.random.normal(size=(K, d))
        
        # for every hyperplane choose one of the items at random
        # we will choose the offset for the hyperplane so that it passes
        # through the selected item (or rather the item's vector)
        random_inds = np.random.randint(n_items, size=K)
        
        # scalar product of every item with the random vectors
        scalar_products = random_vectors.dot(item_vectors.T)
        offsets = scalar_products[range(K), random_inds]

        # for every point for every plane determine 
        # on which side of the plane does the point lie
        # the result is a boolean array of size (n_items, K)
        bits = (scalar_products > offsets.reshape([K, 1])).T

        # for every item encode the sequence of booleans as an integer using binary
        # the result is an integer array of length n_items
        bucket_nums = (bits * (2**np.arange(K))).sum(axis=1)

        # one-hot-encoding on bucket numbers
        sketch = CountVectorizer(analyzer=lambda x: x).fit_transform(bucket_nums.reshape(n_items, 1))
        shallow_sketches.append(sketch)

    return ssp.hstack(shallow_sketches)
    
class EMDEVectorizer(object):
    """A drop-in replacement for CountVectorizer and TfidfVectorizer
    - based on EMDE
    
    The dimensionality of the transformed vectors is not deterministic
    and is at most N * 2**K - but typically much smaller. 
    That is because buckets with no points in them get dropped by CountVectorizer
    """
    def __init__(self, K, N, item2vec, tfidf=False):
        items = list(item2vec.keys())
        item_vectors = np.vstack(list(item2vec.values()))
        
        self.emde_embeddings = EMDE_transform(K, N, item_vectors)
        if tfidf:
            self.vectorizer = TfidfVectorizer(analyzer=lambda x: x, vocabulary=items)
        else:
            self.vectorizer = CountVectorizer(analyzer=lambda x: x, vocabulary=items)
    
    def fit(self, X, y=None):
        # this is only necessary for tfidf=True, otherwise it does nothing
        self.vectorizer.fit(X)
        return self

    def transform(self, X):
        return self.vectorizer.transform(X).dot(self.emde_embeddings)

### Usage

In [2]:
item2vec = {
    'chorizo': np.array([0.2, -0.4, 0.15]),
    'banana': np.array([0.7, -1.2, 2.56]),
    'sourdough': np.array([0.9, 0.1, 0.04])
}

user_baskets = [
    ['chorizo', 'banana'],
    ['sourdough'],
    ['banana', 'banana', 'banana'],
    ['banana', 'chorizo', 'sourdough', 'sourdough']
]

emde = EMDEVectorizer(K=3, N=2, item2vec=item2vec)

users_embedded = emde.transform(user_baskets)
users_embedded.todense()

matrix([[1, 0, 1, 1, 1, 0],
        [0, 1, 0, 0, 0, 1],
        [3, 0, 0, 0, 3, 0],
        [1, 2, 1, 1, 1, 2]], dtype=int64)

In [12]:
from sklearn.preprocessing import normalize
normalize(users_embedded, 'max').todense()

matrix([[1.        , 1.        , 1.        , 1.        ],
        [0.        , 1.        , 0.        , 1.        ],
        [0.        , 1.        , 1.        , 0.        ],
        [0.33333333, 1.        , 0.33333333, 1.        ]])

## Mean-embedding vectorizer
Let's include a vectorizer that takes arithmetic mean of embedding vectors as a simple baseline.

In [2]:
from sklearn.preprocessing import normalize

class MeanEmbeddingVectorizer(object):
    def __init__(self, item2vec, vocabulary=None, tfidf=True):
        self.vocab = vocabulary or item2vec.keys()
        self.item_vectors = np.vstack([item2vec[item] for item in self.vocab])
        if tfidf:
            self.vectorizer = TfidfVectorizer(analyzer=lambda x: x, vocabulary=self.vocab)
        else:
            self.vectorizer = CountVectorizer(analyzer=lambda x: x, vocabulary=self.vocab)
            
    def fit(self, X, y=None):
        self.vectorizer.fit(X)
        return self 

    def transform(self, X):
        item_counts = self.vectorizer.transform(X)
        counts_normed = normalize(item_counts, norm='l1')
        return counts_normed.dot(self.item_vectors)

## Benchmarks
Download some data to test on. We will use a text classifation benchmark.

In [44]:
%%bash
wget http://www.cs.umb.edu/~smimarog/textmining/datasets/r8-train-no-stop.txt
wget http://www.cs.umb.edu/~smimarog/textmining/datasets/r8-test-no-stop.txt
# concatenate train and test files, we'll make our own train-test splits
cat r8-*-no-stop.txt > r8-no-stop.txt

wget http://www.cs.umb.edu/~smimarog/textmining/datasets/20ng-test-no-stop.txt
wget http://www.cs.umb.edu/~smimarog/textmining/datasets/20ng-train-no-stop.txt
cat 20ng-*-no-stop.txt > 20ng-no-stop.txt

--2020-09-28 21:21:51--  http://www.cs.umb.edu/~smimarog/textmining/datasets/r8-train-no-stop.txt
Resolving www.cs.umb.edu (www.cs.umb.edu)... 158.121.106.224
Connecting to www.cs.umb.edu (www.cs.umb.edu)|158.121.106.224|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.cs.umb.edu:443/~smimarog/textmining/datasets/r8-train-no-stop.txt [following]
--2020-09-28 21:21:52--  https://www.cs.umb.edu/~smimarog/textmining/datasets/r8-train-no-stop.txt
Connecting to www.cs.umb.edu (www.cs.umb.edu)|158.121.106.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2537362 (2.4M) [text/plain]
Saving to: ‘r8-train-no-stop.txt.1’

     0K .......... .......... .......... .......... ..........  2%  584K 4s
    50K .......... .......... .......... .......... ..........  4% 1.23M 3s
   100K .......... .......... .......... .......... ..........  6% 11.8M 2s
   150K .......... .......... .......... .......... ..........  8% 1.38M 2s
   200K

Train read the data and train word2vec.

In [3]:
from gensim.models.word2vec import Word2Vec
TRAIN_SET_PATH = "r8-no-stop.txt"

X, y = [], []
with open(TRAIN_SET_PATH, "r") as infile:
    for line in infile:
        label, text = line.split("\t")
        # texts are already tokenized, just split on space
        # in a real case we would use e.g. spaCy for tokenization
        # and maybe remove stopwords etc.
        X.append(text.split())
        y.append(label)
X, y = np.array(X), np.array(y)
print ("total examples %s" % len(y))

# train word2vec on all the texts - both training and test set
# we're not using test labels, just texts so this is fine
model = Word2Vec(X, size=100, window=5, min_count=5, workers=2)
w2v = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

total examples 7674


/home/nadbordrozd/anaconda3/envs/elections/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).


Now for the main event: we make several different types of features:

In [14]:
# simple word counts
counts = normalize(CountVectorizer(analyzer=lambda x: x).fit_transform(X), 'l2')

# embedding vector averaged over all the words per document
mean_vecs = MeanEmbeddingVectorizer(w2v, tfidf=False).fit(X).transform(X)

# and a few configutations of EMDE
emde_8_10 = normalize(EMDEVectorizer(8, 10, w2v, tfidf=False).fit(X).transform(X), 'max')
emde_8_30 = normalize(EMDEVectorizer(8, 30, w2v, tfidf=False).fit(X).transform(X), 'max')
emde_10_10 = normalize(EMDEVectorizer(10, 10, w2v, tfidf=False).fit(X).transform(X), 'max')
emde_10_30 = normalize(EMDEVectorizer(10, 30, w2v, tfidf=False).fit(X).transform(X), 'max')
emde_1_1000 = normalize(EMDEVectorizer(1, 1000, w2v, tfidf=False).fit(X).transform(X), 'max')
emde_1_2000 = normalize(EMDEVectorizer(1, 2000, w2v, tfidf=False).fit(X).transform(X), 'max')
emde_30_1 = normalize(EMDEVectorizer(30, 1, w2v, tfidf=True).fit(X).transform(X), 'max')

In [15]:
feature_sets = {
    'OHE': counts,
    'mean vec': mean_vecs,
    'EMDE K=8 N=10': emde_8_10,
    'EMDE K=8 N=30': emde_8_30,
    'EMDE K=10 N=10': emde_10_10,
    'EMDE K=10 N=30': emde_10_30,
    'EMDE K=1  N=1000': emde_1_1000,
    'EMDE K=1  N=2000': emde_1_2000,
    'EMDE K=30  N=1': emde_30_1,
}

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from time import time

results = {}
mean_results = {}
for name, feats in feature_sets.items():
    print(name, feats.shape[1])
    start = time()
    scores = cross_val_score(LogisticRegression(max_iter=200), feats, y, cv=5)
    end = time()
    print('took %ds' % (end - start))
    print(scores.mean())
    mean_results[name] = scores.mean()
    results[name] = scores

OHE 22931
took 10s
0.9528260620293967
mean vec 100


/home/nadbordrozd/anaconda3/envs/elections/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/nadbordrozd/anaconda3/envs/elections/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit

took 3s
0.9489178617992178
EMDE K=8 N=10 1303
took 17s
0.9644251260250819
EMDE K=8 N=30 4010
took 71s
0.9675526714769248
EMDE K=10 N=10 2554
took 20s
0.9641637752740276
EMDE K=10 N=30 8104
took 82s
0.9678125783011776
EMDE K=1  N=1000 2000


/home/nadbordrozd/anaconda3/envs/elections/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/nadbordrozd/anaconda3/envs/elections/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit

took 223s
0.9644243615932458
EMDE K=1  N=2000 4000


/home/nadbordrozd/anaconda3/envs/elections/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/nadbordrozd/anaconda3/envs/elections/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit

took 441s
0.9679433810820107
EMDE K=30  N=1 3704
took 3s
0.958560235105258


In [18]:
from tabulate import tabulate
final_results = sorted([
    (name, acc, feature_sets[name].shape[1])
    for name, acc in mean_results.items()
], key=lambda x: -x[1])

print(tabulate(final_results, headers=['features', 'accuracy', 'dim']))

features            accuracy    dim
----------------  ----------  -----
EMDE K=1  N=2000    0.967943   4000
EMDE K=10 N=30      0.967813   8104
EMDE K=8 N=30       0.967553   4010
EMDE K=8 N=10       0.964425   1303
EMDE K=1  N=1000    0.964424   2000
EMDE K=10 N=10      0.964164   2554
EMDE K=30  N=1      0.95856    3704
OHE                 0.952826  22931
mean vec            0.948918    100


In [3]:
from gensim.models.word2vec import Word2Vec

TEXTS_PATH = "20ng-no-stop.txt"

X, y = [], []
with open(TEXTS_PATH, "r") as infile:
    for line in infile:
        label, text = line.split("\t")
        # texts are already tokenized, just split on space
        # in a real case we would use e.g. spaCy for tokenization
        # and maybe remove stopwords etc.
        X.append(text.split())
        y.append(label)
X, y = np.array(X), np.array(y)
print ("total examples %s" % len(y))

# train word2vec on all the texts - both training and test set
# we're not using test labels, just texts so this is fine
model = Word2Vec(X, size=100, window=5, min_count=5, workers=2)
w2v = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}

total examples 18821


/home/nadbordrozd/anaconda3/envs/elections/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).


In [4]:
# simple word counts
counts = normalize(CountVectorizer(analyzer=lambda x: x).fit_transform(X), 'l2')

# embedding vector averaged over all the words per document
mean_vecs = MeanEmbeddingVectorizer(w2v, tfidf=False).fit(X).transform(X)

# and a few configutations of EMDE
emde_8_10 = normalize(EMDEVectorizer(8, 10, w2v, tfidf=False).fit(X).transform(X), 'max')
emde_8_30 = normalize(EMDEVectorizer(8, 30, w2v, tfidf=False).fit(X).transform(X), 'max')
emde_10_10 = normalize(EMDEVectorizer(10, 10, w2v, tfidf=False).fit(X).transform(X), 'max')
emde_10_30 = normalize(EMDEVectorizer(10, 30, w2v, tfidf=False).fit(X).transform(X), 'max')
emde_1_1000 = normalize(EMDEVectorizer(1, 1000, w2v, tfidf=False).fit(X).transform(X), 'max')
emde_1_2000 = normalize(EMDEVectorizer(1, 2000, w2v, tfidf=False).fit(X).transform(X), 'max')
emde_30_1 = normalize(EMDEVectorizer(30, 1, w2v, tfidf=True).fit(X).transform(X), 'max')

In [6]:
feature_sets = {
    'OHE': counts,
    'mean vec': mean_vecs,
    'EMDE K=8 N=10': emde_8_10,
    'EMDE K=8 N=30': emde_8_30,
    'EMDE K=10 N=10': emde_10_10,
    'EMDE K=10 N=30': emde_10_30,
    'EMDE K=1  N=1000': emde_1_1000,
    'EMDE K=1  N=2000': emde_1_2000,
    'EMDE K=30  N=1': emde_30_1,
}

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from time import time

results = {}
mean_results = {}
for name, feats in feature_sets.items():
    print(name, feats.shape[1])
    start = time()
    scores = cross_val_score(LogisticRegression(max_iter=200), feats, y, cv=5)
    end = time()
    print('took %ds' % (end - start))
    print(scores.mean())
    mean_results[name] = scores.mean()
    results[name] = scores

OHE 92811
took 197s
0.8258328499674696
mean vec 100


/home/nadbordrozd/anaconda3/envs/elections/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/nadbordrozd/anaconda3/envs/elections/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit

took 15s
0.6195209244495627
EMDE K=8 N=10 2048


/home/nadbordrozd/anaconda3/envs/elections/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/nadbordrozd/anaconda3/envs/elections/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit

took 169s
0.7819983685520053
EMDE K=8 N=30 5850


...

took 816s
0.8374683765822294
EMDE K=1  N=1000 2000


/home/nadbordrozd/anaconda3/envs/elections/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/nadbordrozd/anaconda3/envs/elections/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit

took 690s
0.7537326993831265
EMDE K=1  N=2000 4000


/home/nadbordrozd/anaconda3/envs/elections/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/nadbordrozd/anaconda3/envs/elections/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit

took 1359s
0.764093410276711
EMDE K=30  N=1 20694
took 49s
0.872641859060393


In [8]:
from tabulate import tabulate
final_results = sorted([
    (name, acc, feature_sets[name].shape[1])
    for name, acc in mean_results.items()
], key=lambda x: -x[1])

print(tabulate(final_results, headers=['features', 'accuracy', 'dim']))

features            accuracy    dim
----------------  ----------  -----
EMDE K=30  N=1      0.872642  20694
EMDE K=10 N=30      0.837468  15855
OHE                 0.825833  92811
EMDE K=8 N=30       0.817809   5850
EMDE K=10 N=10      0.809945   5191
EMDE K=8 N=10       0.781998   2048
EMDE K=1  N=2000    0.764093   4000
EMDE K=1  N=1000    0.753733   2000
mean vec            0.619521    100


## More general implementation

In [5]:
from sklearn.pipeline import Pipeline

class Bucketizer(object):
    def __init__(self, K):
        self.K = K
        self.d = None
        self.n_vectors = None
        self.thresholds = None
        self.random_vectors = None

    def fit(self, vectors, y=None):
        self.n_vectors, self.d = vectors.shape
        self.random_vectors = np.random.normal(size=(self.K, self.d))
        scalar_products = self.random_vectors.dot(vectors.T)
        
        random_inds = np.random.randint(self.n_vectors, size=self.K)
        self.thresholds = scalar_products[range(self.K), random_inds]
        return self
    
    def transform(self, vectors):
        n_vectors, dim = vectors.shape
        scalar_products = self.random_vectors.dot(vectors.T)
        bits = (scalar_products > self.thresholds.reshape([self.K, 1])).T.astype('uint8')
        bucket_nums = (bits * (2**np.arange(self.K))).sum(axis=1).astype('uint16')
        return bucket_nums.reshape((n_vectors, 1))


class EMDE(object):
    """This version of EMDE vectorizer can aggregate never before seen vectors
    It also doesn't drop any buckets, even if none of the training vectors fell into the bucket
    """
    def __init__(self, K, N):
        vectorizer = CountVectorizer(analyzer=lambda x: x, 
                                     vocabulary=[i for i in range(2**K)])

        self.bucketizers = [
            Pipeline([('bucketizer', Bucketizer(K)), 
                      ("OHE", vectorizer)])
            for _ in range(N)
        ]
        self.vector_encodings = None
        
    def fit(self, X, y=None):
        """fits the model given an array of vectors
        """
        for b in self.bucketizers:
            b.fit(X)
        return self

    def transform_vectors(self, vectors):
        """takes an array of vectors and transforms them individually returning array of sketches"""
        sketches = np.hstack([b.transform(vectors).todense() 
                              for b in self.bucketizers])
        return sketches


    def transform_vector_set(self, vectors):
        """takes an array of vectors and returns a single sketch representing all of them"""
        return self.transform_vectors(vectors).sum(axis=0)


training_vectors = np.array([
    [-0.1, 0.4, 0.7, 0.3],
    [0.4, 0.12, 0.6, 0.1],
    [0.8, 0.3, 0.5, -0.5],
    [0.1, 0.5, 0.23, 0.6]
])

vectors_to_agg = np.array([
    [0.76, 0.8, -0.6, 0.1],
    [0.23, 0.3, 0.51, -0.5]
])


emde = EMDE(K=3, N=2)
emde.fit(training_vectors)
sketch = emde.transform_vector_set(vectors_to_agg)
sketch

matrix([[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]])